# **TradeCare: Feature Engineering Notebook**   

## Objectives
* Load cleaned Bitcoin OHLCV data
* Engineer technical indicators for ML model
* Create target variables (4-hour prediction horizon)
* Perform correlation analysis with target
* Save engineered features for model training

## Inputs
* **Data Source:** `inputs/datasets/processed/bitcoin_clean.csv`
* **Records:** ~92,180 hourly records (2014-2025)

## Outputs
* **Engineered features CSV:** `inputs/datasets/processed/bitcoin_features.csv`
* **Correlation study** showing which features correlate with profitability
* **Ready-to-train dataset** for classification model

## Features Created
**Price-based:**
* Returns (1h, 4h, 12h, 24h)

**Technical Indicators:**
* RSI (14-period)
* Moving Average (10-period)
* Volume change

**Target Variables:**
* `target_return`: 4-hour ahead price return (%)
* `target_profitable`: Binary (1 = profitable, 0 = not profitable)

## CRISP-DM Phase
Data Preparation → Feature Engineering